In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [2]:
train = pd.read_csv(r"../input/house-prices-advanced-regression-techniques/train.csv")

In [3]:
test = pd.read_csv(r"../input/house-prices-advanced-regression-techniques/test.csv")

In [4]:
test.shape

(1459, 80)

In [5]:
train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [6]:
X = train.iloc[:, :-1].values
y = train.iloc[:, -1].values

In [7]:
X = pd.DataFrame(X)
y = pd.DataFrame(y)

In [8]:
type(X)

pandas.core.frame.DataFrame

In [9]:
type(y)

pandas.core.frame.DataFrame

In [10]:
X.isnull().sum()

0       0
1       0
2       0
3     259
4       0
     ... 
75      0
76      0
77      0
78      0
79      0
Length: 80, dtype: int64

In [11]:
#turn all data to numbers
le = LabelEncoder()
X = X.apply(lambda series: pd.Series(
    LabelEncoder().fit_transform(series[series.notnull()]),
))
X

,0,1,2,3,4,5,6,7,8,9,...,70,71,72,73,74,75,76,77,78,79
0,0,5,3,36.0,327,1,0.0,3,3,0,...,0,0,0.0,2.0,2.0,0,1,2,8,4
1,1,0,3,51.0,498,1,1.0,3,3,0,...,0,0,1.0,1.0,2.0,0,4,1,8,4
2,2,5,3,39.0,702,1,1.0,0,3,0,...,0,0,2.0,0.0,2.0,0,8,2,8,4
3,3,6,3,31.0,489,1,0.0,0,3,0,...,0,0,0.0,2.0,2.0,0,1,0,8,0
4,4,5,3,55.0,925,1,1.0,0,3,0,...,0,0,2.0,0.0,2.0,0,11,2,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1455,5,3,NaN,267,1,NaN,3,3,0,...,0,0,NaN,NaN,NaN,0,7,1,8,4
1456,1456,0,3,NaN,866,1,NaN,3,3,0,...,0,0,NaN,NaN,NaN,0,1,4,8,4
1457,1457,6,3,NaN,415,1,NaN,3,3,0,...,0,0,NaN,NaN,NaN,17,4,4,8,4
1458,1458,0,3,NaN,505,1,NaN,3,3,0,...,0,0,NaN,NaN,NaN,0,3,4,8,4


In [12]:
#mean in place of NaNs
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X.iloc[:, :])
X.iloc[:, :] = imputer.transform(X.iloc[:, :])

In [13]:
#scaling data in order to use in linear regression
sc = StandardScaler()
X.iloc[:, :] = sc.fit_transform(X.iloc[:, :])

In [14]:
le = LabelEncoder()
train = train.apply(lambda series: pd.Series(
    LabelEncoder().fit_transform(series[series.notnull()]),
))

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(train.iloc[:, :])
train.iloc[:, :] = imputer.transform(train.iloc[:, :])

sc = StandardScaler()
train.iloc[:, :] = sc.fit_transform(train.iloc[:, :])

In [15]:
# corr = train.corr()
# plt.figure(figsize=(81, 81))
# sns.set(font_scale=4)
# sns.heatmap(corr, cbar=True, square= True, fmt='.1f', annot=True, annot_kws={'size':35}, cmap='Oranges')

In [16]:
# comparing different types of train data to get the best result(data with the most correlation 
# / all data including not integer ones / only integer data)

# X = X[[17, 46, 54, 47, 19, 20, 56, 59, 38, 43, 61, 62, 26, 27, 53, 29, 34, 67, 44, 40, 39]]
# #X = data["ZN"].values.reshape((-1,1))
# y = y[0]
# X = X[[1, 3, 4, 17, 18, 19, 20, 26, 34, 36, 37, 38, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 56, 59, 61, 62, 66, 67
#        , 68, 69, 70, 71, 75, 76, 77]]

In [17]:
# determining the test part of train data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.35, random_state = 1)

In [18]:
# Create a Linear regressor
mlr = LinearRegression()

# Train the model using the training sets 
mlr.fit(X_train, y_train)

LinearRegression()

In [19]:
y_pred = mlr.predict(X_train)

In [20]:
print('R^2:',metrics.r2_score(y_train, y_pred))
print('MAE:',metrics.mean_absolute_error(y_train, y_pred))
print('MSE:',metrics.mean_squared_error(y_train, y_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train, y_pred)))

R^2: 0.8409028326816361
MAE: 19545.24331000545
MSE: 957574721.4384241
RMSE: 30944.704255145567


In [21]:
y_pred2 = mlr.predict(X_test)
print('R^2:',metrics.r2_score(y_test, y_pred2))
print('MAE:',metrics.mean_absolute_error(y_test, y_pred2))
print('MSE:',metrics.mean_squared_error(y_test, y_pred2))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_pred2)))

R^2: 0.8405452100905784
MAE: 20872.313562993295
MSE: 1090726828.9880948
RMSE: 33026.153711688785


In [22]:
X_test_test = test.iloc[:, :].values

In [23]:
X_test_test = pd.DataFrame(X_test_test)

In [24]:
#turn all data to numbers
le = LabelEncoder()
X_test_test = X_test_test.apply(lambda series: pd.Series(
    LabelEncoder().fit_transform(series[series.notnull()]),
))
X_test_test

,0,1,2,3,4,5,6,7,8,9,...,70,71,72,73,74,75,76,77,78,79
0,0,0,2.0,56.0,777,1,1.0,3,3,0.0,...,18,0,0.0,2.0,0.0,0,5,4,8.0,4
1,1,0,3.0,57.0,964,1,1.0,0,3,0.0,...,0,0,0.0,2.0,2.0,24,5,4,8.0,4
2,2,5,3.0,50.0,947,1,1.0,0,3,0.0,...,0,0,1.0,0.0,2.0,0,2,4,8.0,4
3,3,5,3.0,54.0,587,1,1.0,0,3,0.0,...,0,0,NaN,2.0,2.0,0,5,4,8.0,4
4,4,11,3.0,19.0,118,1,1.0,0,1,0.0,...,27,0,NaN,0.0,2.0,0,0,4,8.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,1454,13,3.0,NaN,19,1,NaN,3,3,0.0,...,0,0,NaN,NaN,NaN,0,5,0,8.0,4
1455,1455,13,NaN,NaN,17,1,NaN,3,3,0.0,...,0,0,NaN,NaN,NaN,0,3,0,8.0,0
1456,1456,0,NaN,NaN,1069,1,NaN,3,3,0.0,...,0,0,NaN,NaN,NaN,0,8,0,8.0,0
1457,1457,9,NaN,NaN,647,1,NaN,3,3,NaN,...,0,0,NaN,NaN,NaN,12,6,0,8.0,4


In [25]:
#mean in place of NaNs
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X_test_test.iloc[:, :])
X_test_test.iloc[:, :] = imputer.transform(X_test_test.iloc[:, :])

In [26]:
list(map(tuple, np.where(np.isnan(X_test_test))))

[(), ()]

In [27]:
X_test_test

,0,1,2,3,4,5,6,7,8,9,...,70,71,72,73,74,75,76,77,78,79
0,0.0,0.0,2.000000,56.000000,777.0,1.0,1.000000,3.0,3.0,0.0,...,18.0,0.0,0.000000,2.000000,0.000000,0.0,5.0,4.0,8.000000,4.0
1,1.0,0.0,3.000000,57.000000,964.0,1.0,1.000000,0.0,3.0,0.0,...,0.0,0.0,0.000000,2.000000,2.000000,24.0,5.0,4.0,8.000000,4.0
2,2.0,5.0,3.000000,50.000000,947.0,1.0,1.000000,0.0,3.0,0.0,...,0.0,0.0,1.000000,0.000000,2.000000,0.0,2.0,4.0,8.000000,4.0
3,3.0,5.0,3.000000,54.000000,587.0,1.0,1.000000,0.0,3.0,0.0,...,0.0,0.0,0.333333,2.000000,2.000000,0.0,5.0,4.0,8.000000,4.0
4,4.0,11.0,3.000000,19.000000,118.0,1.0,1.000000,0.0,1.0,0.0,...,27.0,0.0,0.333333,0.000000,2.000000,0.0,0.0,4.0,8.000000,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,1454.0,13.0,3.000000,44.435877,19.0,1.0,0.345794,3.0,3.0,0.0,...,0.0,0.0,0.333333,1.396552,1.843137,0.0,5.0,0.0,8.000000,4.0
1455,1455.0,13.0,3.026804,44.435877,17.0,1.0,0.345794,3.0,3.0,0.0,...,0.0,0.0,0.333333,1.396552,1.843137,0.0,3.0,0.0,8.000000,0.0
1456,1456.0,0.0,3.026804,44.435877,1069.0,1.0,0.345794,3.0,3.0,0.0,...,0.0,0.0,0.333333,1.396552,1.843137,0.0,8.0,0.0,8.000000,0.0
1457,1457.0,9.0,3.026804,44.435877,647.0,1.0,0.345794,3.0,3.0,0.0,...,0.0,0.0,0.333333,1.396552,1.843137,12.0,6.0,0.0,8.000000,4.0


In [28]:
#scaling data in order to use in linear regression
sc = StandardScaler()
X_test_test.iloc[:, :] = sc.fit_transform(X_test_test.iloc[:, :])

In [29]:
X_test_test

,0,1,2,3,4,5,6,7,8,9,...,70,71,72,73,74,75,76,77,78,79
0,-1.730864,-0.981333,-1.501888,0.595020,0.836391,0.06426,5.079067e+00,0.742620,0.31870,0.0,...,1.151627,-0.057729,-1.559380e+01,1.674872e+00,-1.972031e+01,-0.162373,-0.038281,1.713905,3.249845e-01,0.201383
1,-1.728490,-0.981333,-0.039206,0.646474,1.440233,0.06426,5.079067e+00,-1.384637,0.31870,0.0,...,-0.287195,-0.057729,-1.559380e+01,1.674872e+00,1.678324e+00,9.674419,-0.038281,1.713905,3.249845e-01,0.201383
2,-1.726115,0.160790,-0.039206,0.286296,1.385338,0.06426,5.079067e+00,-1.384637,0.31870,0.0,...,-0.287195,-0.057729,3.118760e+01,-3.876133e+00,1.678324e+00,-0.162373,-1.140614,1.713905,3.249845e-01,0.201383
3,-1.723741,0.160790,-0.039206,0.492112,0.222861,0.06426,5.079067e+00,-1.384637,0.31870,0.0,...,-0.287195,-0.057729,-2.596890e-15,1.674872e+00,1.678324e+00,-0.162373,-0.038281,1.713905,3.249845e-01,0.201383
4,-1.721367,1.531337,-0.039206,-1.308778,-1.291588,0.06426,5.079067e+00,-1.384637,-2.53396,0.0,...,1.871038,-0.057729,-2.596890e-15,-3.876133e+00,1.678324e+00,-0.162373,-1.875504,1.713905,3.249845e-01,0.201383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,1.721367,1.988186,-0.039206,0.000000,-1.611269,0.06426,-8.619457e-16,0.742620,0.31870,0.0,...,-0.287195,-0.057729,-2.596890e-15,6.162853e-16,-4.751451e-15,-0.162373,-0.038281,-1.359958,3.249845e-01,0.201383
1455,1.723741,1.988186,0.000000,0.000000,-1.617727,0.06426,-8.619457e-16,0.742620,0.31870,0.0,...,-0.287195,-0.057729,-2.596890e-15,6.162853e-16,-4.751451e-15,-0.162373,-0.773170,-1.359958,3.249845e-01,-3.589818
1456,1.726115,-0.981333,0.000000,0.000000,1.779289,0.06426,-8.619457e-16,0.742620,0.31870,0.0,...,-0.287195,-0.057729,-2.596890e-15,6.162853e-16,-4.751451e-15,-0.162373,1.064053,-1.359958,3.249845e-01,-3.589818
1457,1.728490,1.074488,0.000000,0.000000,0.416607,0.06426,-8.619457e-16,0.742620,0.31870,0.0,...,-0.287195,-0.057729,-2.596890e-15,6.162853e-16,-4.751451e-15,4.756023,0.329164,-1.359958,3.249845e-01,0.201383


In [30]:
# X_test_test = X_test_test[[17, 46, 54, 47, 19, 20, 56, 59, 38, 43, 61, 62, 26, 27, 53, 29, 34, 67, 44, 40, 39]]
# X_test_test = X_test_test[[1, 3, 4, 17, 18, 19, 20, 26, 34, 36, 37, 38, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 56, 59, 61, 62, 66, 67
#        , 68, 69, 70, 71, 75, 76, 77]]

In [31]:
# pd.DataFrame(X_test_test).to_excel(r"E:\data\ML-utech\test.xlsx", index=False)

In [32]:
y_ans = mlr.predict(X_test_test)

In [33]:
answer = pd.DataFrame(y_ans)

In [34]:
answer.columns = ['SalePrice']
answer

,SalePrice
0,134037.247088
1,185599.250769
2,176266.458783
3,207355.320574
4,189301.650501
...,...
1454,80748.206927
1455,106441.606800
1456,136094.808951
1457,141250.886852


In [35]:
ending_id = pd.DataFrame(test.iloc[:, 0].values)
ending_id.columns = ['Id']
answer = pd.concat([ending_id, answer], axis = 1)

In [36]:
answer.to_csv('submission.csv', index=False)

In [37]:
print(answer)

        Id      SalePrice
0     1461  134037.247088
1     1462  185599.250769
2     1463  176266.458783
3     1464  207355.320574
4     1465  189301.650501
...    ...            ...
1454  2915   80748.206927
1455  2916  106441.606800
1456  2917  136094.808951
1457  2918  141250.886852
1458  2919  250661.139567

[1459 rows x 2 columns]
